In [2]:
import pandas as pd
import numpy as np
import json
from os import makedirs, listdir
from os.path import join, exists

In [3]:
#extract all fake sources from signal media
df_opensources_co = pd.read_csv('data/opensources_co.csv', sep=',') 

In [4]:
df_opensources_co.rename(columns = {'Unnamed: 0': 'url'}, inplace=True)

In [5]:
df_opensources_co.drop(['Unnamed: 5'], inplace=True, axis=1)

In [6]:
df_opensources_co = df_opensources_co.sort_values(by = ['url'])

In [7]:
df_opensources_co.groupby('type').count()

,url,2nd type,3rd type,Source Notes (things to know?)
type,,,,
Conspiracy,1,1,1,1
bias,133,92,30,29
clickbait,35,27,5,2
conspiracy,121,65,17,17
fake,235,73,21,132
fake,1,0,0,1
fake news,1,1,0,1
hate,29,12,3,13
junksci,32,15,1,0


In [8]:
# there are two diff spelling for unreliable
mask = ((df_opensources_co['type'] == 'political' ) & (df_opensources_co['2nd type'] != 'unreliable') & (df_opensources_co['2nd type'] != 'unrealiable')) | (df_opensources_co['type'] == 'reliable') | (df_opensources_co['type'] == 'clickbait') | (df_opensources_co['type'] == 'hate')

df_opensources_co = df_opensources_co[~mask]    
df_opensources_co.to_csv('data/fake_sources.csv', sep=',', header=True)

In [9]:
#read data from creible and fake source
fake_src = df_opensources_co['url'].apply(lambda x: x.split('.')[0].lower())
fake_src = list(fake_src)

In [10]:
data_real = []
data_fake = []
cred_src_list = ['abcnews', 'bloomberg', 'buzzfeed', 'cbsnews', 'cnn','cbcnews','bbcnews', 'bbcsport', 'cnbc', 'espn', 
                 'foxnews', 'foxsports', 'msnbc', 'nbcnews', 'reuters', 'news24', 
                 'thenewyorktimes', 'usatoday', 'wired', 'vicenews', 'thewashingtontimes', 
                 'thewashingtonpost', 'techcrunch', 'washingtonpost']

with open('signalmedia-1m.jsonl', 'r') as f:
    for line in f:
        jobj = json.loads(line)
        s = jobj['source'].replace(' ','').replace("'","").replace('-','').lower()
        if s in fake_src and s not in cred_src_list:
            jobj['label'] = 1
            data_fake.append(jobj)
        elif s in cred_src_list and s not in fake_src:
            jobj['label'] = 0
            data_real.append(jobj)
        else:
            pass

print(len(data_real))
print(len(data_fake))

9280
3212


In [11]:
data_real = pd.DataFrame(data_real)
data_fake = pd.DataFrame(data_fake)

In [12]:
grouped = data_real.groupby('source').count()
grouped.sort_values('content', ascending=False)

,content,id,label,media-type,published,title
source,,,,,,
Reuters,3898,3898,3898,3898,3898,3898
Bloomberg,2833,2833,2833,2833,2833,2833
CNBC,948,948,948,948,948,948
CNN,595,595,595,595,595,595
News24,556,556,556,556,556,556
BuzzFeed,247,247,247,247,247,247
TechCrunch,114,114,114,114,114,114
WIRED,89,89,89,89,89,89


In [13]:
grouped = data_fake.groupby('source').count()
grouped.sort_values('content', ascending=False)

,content,id,label,media-type,published,title
source,,,,,,
Before It's News,1084,1084,1084,1084,1084,1084
Before Its News,982,982,982,982,982,982
Zero Hedge,149,149,149,149,149,149
Washington Examiner,79,79,79,79,79,79
Infowars,67,67,67,67,67,67
Western Journalism,61,61,61,61,61,61
Global Research,60,60,60,60,60,60
Right Wing News,55,55,55,55,55,55
Weasel Zippers,45,45,45,45,45,45


In [16]:
data_real.groupby('source').count()

,content,id,label,media-type,published,title
source,,,,,,
Bloomberg,2833,2833,2833,2833,2833,2833
BuzzFeed,247,247,247,247,247,247
CNBC,948,948,948,948,948,948
CNN,595,595,595,595,595,595
News24,556,556,556,556,556,556
Reuters,3898,3898,3898,3898,3898,3898
TechCrunch,114,114,114,114,114,114
WIRED,89,89,89,89,89,89


In [17]:
data_real = data_real.sample(frac=0.5)

In [18]:
data_real.groupby('source').count()

,content,id,label,media-type,published,title
source,,,,,,
Bloomberg,1439,1439,1439,1439,1439,1439
BuzzFeed,118,118,118,118,118,118
CNBC,453,453,453,453,453,453
CNN,306,306,306,306,306,306
News24,276,276,276,276,276,276
Reuters,1943,1943,1943,1943,1943,1943
TechCrunch,56,56,56,56,56,56
WIRED,49,49,49,49,49,49


In [21]:
data_df = pd.concat([data_real, data_fake], axis=0)

In [22]:
data_df = data_df.sample(frac=1).reset_index(drop=True)

In [23]:
data_df.groupby('label').count()

,content,id,media-type,published,source,title
label,,,,,,
0,4640,4640,4640,4640,4640,4640
1,3212,3212,3212,3212,3212,3212


In [24]:
data_df.to_pickle('data/all_data.pkl')